In [1]:
from google.colab import drive
drive.mount('/content/drive')

drive_path = "/content/drive/My Drive/Colab Notebooks/hypefyn/"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# %%
import json
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import HashingVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import average_precision_score, confusion_matrix, classification_report

In [0]:
# %%
with open(drive_path+"data/sentiment140/train_clean_joint.json",'r') as t:
    train = json.load(t)
with open(drive_path+"data/sentiment140/val_clean_joint.json",'r') as v:
    test = json.load(v)

# %%
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer()),  # integer counts to weighted TF-IDF scores
    ('classifier', MultinomialNB()),  # train on TF-IDF vectors w/ Naive Bayes classifier
])


In [0]:
def dict_extract(d):
    labels = []
    texts = []
    for k in d.keys():
        tweet = d[k]
        label = tweet['label']
        text = tweet['tokens']
        labels.append(label)
        texts.append(text)
    return texts, labels

In [0]:
train_text, train_label = dict_extract(train)
test_text, test_label = dict_extract(test)

In [30]:
pipeline.fit(train_text, train_label)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, use_idf=True,
                                 vocabulary=None)),
                ('classifier',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

In [0]:
test_pred = pipeline.predict(test_text)

In [36]:
print(classification_report(test_label, test_pred),
      confusion_matrix(test_label, test_pred))

              precision    recall  f1-score   support

           0       0.63      0.79      0.70       177
           2       0.00      0.00      0.00       139
           4       0.51      0.78      0.62       182

    accuracy                           0.57       498
   macro avg       0.38      0.52      0.44       498
weighted avg       0.41      0.57      0.48       498
 [[140   0  37]
 [ 41   0  98]
 [ 40   0 142]]


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
with open(drive_path+"data/tweets_clean_joint.json",'r') as tw:
    tweets = json.load(tw)

tweets_text, tweets_keyword = dict_extract(tweets)

In [0]:
import pandas as pd

tweets_data = pd.read_csv(drive_path+"data/all_tweets.csv")
tweets_tweet = tweets_data.text.to_list()

In [0]:
tweets_pred = pipeline.predict(tweets_text)

In [0]:
tweets_sent = ["Pos" if sent==4 else "Neg" for sent in tweets_pred]
tweets_out = pd.concat([tweets_data, pd.DataFrame(tweets_sent,columns=["Prediction"])],axis=1)

In [0]:
tweets_out.to_csv(drive_path+"data/tweets_pred.csv")